In [12]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from config import BaseConfig
config = BaseConfig()

In [13]:
data_df = pd.read_csv(config.data_csv_path)

In [15]:
data_df['current_combined'] = data_df['current_province'] + data_df['current_city'] + data_df['current_county']

In [16]:
from utils.loading import get_json

level = get_json(config.level_path)

In [17]:
def fill_current(df: pd.DataFrame, level: list) -> pd.DataFrame:
    # Initialize the columns
    df['current_province_code'] = None
    df['current_city_code'] = None
    df['current_county_code'] = None
    
    # Iterate over the rows in the DataFrame
    for idx, row in df.iterrows():
        row_province = row['current_province']
        row_city = row['current_city']
        row_county = row['current_county']
        
        # Iterate over the `level` list (assuming it's a list of dicts)
        for province in level:
            if row_province != province['name']:
                continue

            df.at[idx, 'current_province_code'] = province['province']
            # case 1: city is province, 重庆 data is malformed and is treated as a normal province
            if row_city == row_province:
                df.at[idx, 'current_city_code'] = province['children'][0]['city']
                for county in province['children']:
                    if county['name'] == row_county:
                        df.at[idx, 'current_county_code'] = county['area']
                        break
            else:
                for city in province['children']:
                    if row_city != city['name']:
                        continue

                    df.at[idx, 'current_city_code'] = city['city']
                    for county in city['children']:
                        if row_county != county['name']:
                            continue

                        df.at[idx, 'current_county_code'] = county['area']
                        break

    return df


In [18]:
# Military bases will be None
filled_df = fill_current(data_df.copy(), level=level)

In [19]:
unfound_chongqing_rows = (filled_df['current_city'] == '重庆市') & (filled_df['current_county_code'].isna())

# chongqing has missing counties from the dataset
filled_df[unfound_chongqing_rows]['current_combined'].nunique()

5

In [20]:
unfound_cities_rows = (filled_df['current_city_code'].isna())


filled_df[unfound_cities_rows]['current_combined'].nunique()

127

In [21]:
found_citites_but_unfound_neighborhood_rows = (~filled_df['current_city_code'].isna()) & (filled_df['current_county_code'].isna())

filled_df[found_citites_but_unfound_neighborhood_rows]['current_combined'].nunique()

153

In [22]:
data_df['hometown_lon'].isna().sum() / len(data_df), data_df['hometown_lat'].isna().sum() / len(data_df)


(np.float64(0.10353611116013389), np.float64(0.10353611116013389))

In [23]:
data_df['first_lon'].isna().sum() / len(data_df), data_df['first_lat'].isna().sum() / len(data_df)

(np.float64(0.19997764561236314), np.float64(0.19997764561236314))

In [24]:
data_df['current_lon'].isna().sum() / len(data_df), data_df['current_lat'].isna().sum() / len(data_df)

(np.float64(0.13364982440040238), np.float64(0.13364982440040238))

In [25]:
hometown_lon_lat_missing = filled_df['hometown_lon'].isna() | filled_df['hometown_lat'].isna()
first_lon_lat_missing = filled_df['first_lon'].isna() | filled_df['first_lat'].isna()
current_lon_lat_missing = filled_df['current_lon'].isna() | filled_df['current_lat'].isna()

In [26]:
filled_df[hometown_lon_lat_missing]['hometown'].nunique()

745

In [27]:
filled_df[first_lon_lat_missing]['first_flow_location'].nunique()

696

In [28]:
filled_df[current_lon_lat_missing]['current_combined'].nunique()

271

In [29]:
# list of all locations with missing ids
locations_missing_ids = np.unique(np.concatenate([
    filled_df[unfound_chongqing_rows]['current_combined'].unique(), 
    filled_df[unfound_cities_rows]['current_combined'].unique(),
    filled_df[found_citites_but_unfound_neighborhood_rows]['current_combined'].unique()
]))

locations_missing_coordiantes = np.unique(np.concatenate([
    filled_df[hometown_lon_lat_missing]['hometown'].unique(),
    filled_df[first_lon_lat_missing]['first_flow_location'].unique(),
    filled_df[current_lon_lat_missing]['current_combined'].unique()
]))

locations_to_be_queried = np.unique(np.concatenate([locations_missing_ids, locations_missing_coordiantes]))

len(locations_missing_ids), len(locations_missing_coordiantes), len(locations_to_be_queried)

(280, 1018, 1030)

In [30]:
total_mask = (unfound_chongqing_rows) | (unfound_cities_rows) | (found_citites_but_unfound_neighborhood_rows) | (hometown_lon_lat_missing) | (first_lon_lat_missing) | (current_lon_lat_missing)
total_geo_mask = (hometown_lon_lat_missing) | (first_lon_lat_missing) | (current_lon_lat_missing)
total_ids_mask = (unfound_chongqing_rows) | (unfound_cities_rows) | (found_citites_but_unfound_neighborhood_rows)

len(filled_df[total_mask]) / len(filled_df), len(filled_df[total_geo_mask]) / len(filled_df), len(filled_df[total_ids_mask])/ len(filled_df)

(0.3441399149356723, 0.34084558412603166, 0.1364735365229515)

In [31]:
cleaned_df = filled_df[~total_mask]

In [32]:
for column in cleaned_df.columns:
    if cleaned_df[column].isna().sum() != 0:
        print(column, cleaned_df[column].isna().sum())

average_family_cost_per_month 6
average_family_income_per_month 4
how_long_to_stay 18856


In [33]:
def process_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['hometown_province_code'] = df['hometown_code'].apply(lambda x: str(x)[:2])
    df['hometown_city_code'] = df['hometown_code'].apply(lambda x: str(x)[2:4])
    df['hometown_county_code'] = df['hometown_code'].apply(lambda x: str(x)[4:6])
    df['first_flow_province_code'] = df['first_flow_code'].apply(lambda x: str(x)[:2])
    df['first_flow_city_code'] = df['first_flow_code'].apply(lambda x: str(x)[2:4])
    df['first_flow_county_code'] = df['first_flow_code'].apply(lambda x: str(x)[4:6])
    df['current_code'] = df['current_province_code'].astype(str) + df['current_city_code'].astype(str) + df['current_county_code'].astype(str)

    columns_to_keep = [
        'hometown_code',
        'hometown_province_code',
        'hometown_city_code',
        'hometown_county_code',
        'hometown_lon',
        'hometown_lat',

        'first_flow_code',
        'first_flow_province_code',
        'first_flow_city_code',
        'first_flow_county_code',
        'first_lon',
        'first_lat',
        'year_first_flow',
        'month_first_flow',

        'current_code',
        'current_province_code',
        'current_city_code',
        'current_county_code',
        'current_lon',
        'current_lat',
        'year_current_flow',
        'month_current_flow',

        'gender',
        'edu_level',
        'average_family_cost_per_month',
        'average_family_income_per_month',
        'num_flows_total',
        'if_stay',
        'if_change_household_local',
        'how_long_to_stay',
        
    ]
    # drop insignificant rows of missing data
    df = df.dropna(subset=['average_family_cost_per_month', 'average_family_income_per_month'])
    df = df[columns_to_keep]
    return df

In [34]:
cleaned_df = process_df(cleaned_df)

In [35]:
cleaned_df.to_csv(config.data_folder / 'cleaned_data.csv', index=False)

In [37]:
cleaned_df['edu_level'].value_counts()

edu_level
3    48382
4    24634
2    15616
5    12051
6     7288
1     2923
7      585
Name: count, dtype: int64